<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [11]:
!conda install -c conda-forge wikipedia --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium --yes # uncomment this line if you haven't completed the Foursquare API lab

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import wikipedia as wp


print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    wikipedia: 1.4.0-py35_0 conda-forge

wikipedia-1.4. 100% |################################| Time: 0:00:00  17.18 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    folium: 0.5.0-py_0 conda-forge --> 0.8.3-py_0 conda-forge

folium-0.8.3-p 100% |################################| Time: 0:00:00  27.48 MB/s
Libraries imported.


Use the Notebook to build the code to scrape the following Wikipedia page, 

In [12]:
html="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [13]:
df = pd.read_html(html,header=0)[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [14]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So for the 9th cell in the table on the Wikipedia page,
the value of the Borough and the Neighborhood columns will be Queen's Park.

In [15]:
# sorting on Post Code
df=df.sort_values(by=['Postcode'])
print(df.head(10))

j=df.Postcode.count()-1
print("Reoords to run", j)

i=0
while i<j:
    if ((df.iloc[i]['Postcode'])==(df.iloc[i+1]['Postcode'])):
        df.iloc[i+1]['Neighbourhood'] =  df.iloc[i]['Neighbourhood']  + ", " + df.iloc[i+1]['Neighbourhood']
# Mark Records for deletion
        df.iloc[i]['Neighbourhood'] = "DLELTE"
    i=i+1

# with records marked for deletion
print(df.shape)

#Delete the records marked for deletion,
df = df[df['Neighbourhood'] != 'DLELTE'].reset_index(drop=True)
print(df.head)

   Postcode      Borough   Neighbourhood
8       M1B  Scarborough           Rouge
9       M1B  Scarborough         Malvern
23      M1C  Scarborough      Port Union
22      M1C  Scarborough      Rouge Hill
21      M1C  Scarborough  Highland Creek
33      M1E  Scarborough       Guildwood
34      M1E  Scarborough     Morningside
35      M1E  Scarborough       West Hill
39      M1G  Scarborough          Woburn
43      M1H  Scarborough       Cedarbrae
Reoords to run 211
(212, 3)
<bound method NDFrame.head of     Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T   

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [16]:
print(df.shape)

(103, 3)


Read GEO data

In [18]:
geoData = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
print(geoData.head())
print(len(df.index))

#for i in range (0, len(df.index)):
print(df.loc[0]['Postcode'])

#df2=df1.loc[df1['Postal Code'] == df.loc[i]['Postcode']]
geoData.rename(columns={"Postal Code": "Postcode"}, inplace=True)
print(geoData.head())

#result = pd.merge(df,df1[['Latitude', 'Longitude']],left_on='Postcode', right_on="Postal Code", how="left")
mergedDF = pd.merge(df,geoData, on='Postcode')

print(mergedDF.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
103
M1B
  Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476
  Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Port Union, Rouge Hill, Highland Creek  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  


In [19]:
import folium
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create map of Tornto  using latitude and longitude values

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Postcode, Borough, Neighbourhood in zip(mergedDF['Latitude'], mergedDF['Longitude'],
                                                       mergedDF['Postcode'], mergedDF['Borough'],
                                                       mergedDF['Neighbourhood']):
# label = '{}, {}, {}'.format(Postcode, Borough, Neighbourhood)
    label = '{}, {}'.format(Postcode, Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        #parse_html=False
        ).add_to(map_toronto)

map_toronto

Import request

In [21]:
import requests

Setup Four square API

In [23]:
CLIENT_ID = '2G3RU0HVXD2R2MVA0VS3VOXRUB3OBP2SAOXL1E5CNCQZKAPS' # your Foursquare ID
CLIENT_SECRET = '3WJ2JCHLUD52FNDERTYH0FFHVROWIRRM3QJ1Z0JWW5THTXCF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentials Setup done')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials Setup done


Define radius to search near by place and count of total location 

In [26]:
radius = 500
limit = 100

Function to fetch the data from Fours sqare API 

In [27]:
def getNearbyVenues(codes, names, latitudes, longitudes, radius=500):
    venues_list = []
    for code, name, lat, lng in zip(codes, names, latitudes, longitudes):
        print(code, name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}\
            &v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            limit)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']



        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name,
            lat,
            lng,
            #            v['venue']['code'],
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                             'Borough',
                             'Latitude',
                             'Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return (nearby_venues)

Details of nearby places for addtional research

In [28]:
toronto_venues = getNearbyVenues(codes=mergedDF['Postcode'],
                                  names=mergedDF['Borough'],
                                  latitudes=mergedDF['Latitude'],
                                  longitudes=mergedDF['Longitude']
                                   )


print(toronto_venues.shape)
print(toronto_venues.head())


toronto_venues.groupby(['Postal Code', 'Borough']).count()

M1B Scarborough
M1C Scarborough
M1E Scarborough
M1G Scarborough
M1H Scarborough
M1J Scarborough
M1K Scarborough
M1L Scarborough
M1M Scarborough
M1N Scarborough
M1P Scarborough
M1R Scarborough
M1S Scarborough
M1T Scarborough
M1V Scarborough
M1W Scarborough
M1X Scarborough
M2H North York
M2J North York
M2K North York
M2L North York
M2M North York
M2N North York
M2P North York
M2R North York
M3A North York
M3B North York
M3C North York
M3H North York
M3J North York
M3K North York
M3L North York
M3M North York
M3N North York
M4A North York
M4B East York
M4C East York
M4E East Toronto
M4G East York
M4H East York
M4J East York
M4K East Toronto
M4L East Toronto
M4M East Toronto
M4N Central Toronto
M4P Central Toronto
M4R Central Toronto
M4S Central Toronto
M4T Central Toronto
M4V Central Toronto
M4W Downtown Toronto
M4X Downtown Toronto
M4Y Downtown Toronto
M5A Downtown Toronto
M5B Downtown Toronto
M5C Downtown Toronto
M5E Downtown Toronto
M5G Downtown Toronto
M5H Downtown Toronto
M5J Downtow

,,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,Borough,,,,,,
M1B,Scarborough,2,2,2,2,2,2
M1C,Scarborough,2,2,2,2,2,2
M1E,Scarborough,7,7,7,7,7,7
M1G,Scarborough,3,3,3,3,3,3
M1H,Scarborough,7,7,7,7,7,7
M1J,Scarborough,1,1,1,1,1,1
M1K,Scarborough,4,4,4,4,4,4
M1L,Scarborough,10,10,10,10,10,10
M1M,Scarborough,3,3,3,3,3,3
